In [91]:
def printLine(s, pad, offset, length):
    t = [pad for _ in range(length+1)]
    t[offset:offset+len(s)+1] = s
    return t

In [90]:
def gsSkip(p, gs):
    len_p = len(p)
    len_gs = len(gs)
    offset = len_p - len_gs - 1
    skip = 0
    for i in range(offset, -1, -1): # alignments
        s = p[max(0, i-len_gs+1):i+1]
        len_s = len(s)
        for j in range(len_s-1, -1, -1): # bases
            if (s[j] != gs[j+len_gs-len_s]):
                break
            else:
                if (j==0):
                    skip = offset + len_gs - i - 1
                    #print("i: %d, j: %d, offset: %d, gs: %d" % (i, j, offset, len_gs))
        if skip > 0:
            break
    #print("break: %d" % skip)
    return skip

In [89]:
def bcSkip(p, offset, bc):
    for j in range(offset-1, -1, -1):
        #print(p, offset, bc, j)
        if (p[j] == bc):
            return offset-j-1
    return offset

In [109]:
def bmSteps(p, t):
    print("%s|" % ' '.join([t[i] for i in range(len(t))]))
    algn = 0
    while algn < len(t)-len(p)+1:
        s = printLine(p, ' ', algn, len(t))
        for offset in range(len(p)-1, -1, -1):
            gs_skip = 0
            bc_skip = 0
            gs = ''
            bc = ''
            #print(algn, offset, t[offset+algn], s[offset+algn])
            if t[offset+algn] != s[offset+algn]:
                bc = t[offset+algn]
                bc_skip = bcSkip(p, offset, bc)
                gs = p[offset+1:]
                if (len(gs)>0):
                    gs_skip = gsSkip(p, gs)
                algn += max(bc_skip, gs_skip)
                print("%s| \x1b[31mbc: %s, %d\x1b[0m, \x1b[32mgs: %s, %d\x1b[0m" % (' '.join([s[i] for i in range(len(s))]), 
                                         bc, 
                                         bc_skip,
                                         ''.join([gs[i] for i in range(len(gs))]),
                                         gs_skip
                                        ))
                break
            else:
                if (bc == '') and (offset == 0):
                    print("\x1b[32m%s| MATCH\x1b[0m" % ' '.join([s[i] for i in range(len(s))]))
                    break
        algn += 1

In [107]:
p = 'GTAGCGGCG'
t = 'GTTATAGCTGATCGCGGCGTAGCGGCGAA'
bmSteps(p, t)

G T T A T A G C T G A T C G C G G C G T A G C G G C G A A|
G T A G C G G C G                                        | bc: T, 6, gs: , 0
              G T A G C G G C G                          | bc: C, 0, gs: GCG, 2
                    G T A G C G G C G                    | bc: C, 2, gs: GCGGCG, 7
                                    G T A G C G G C G    | MATCH
                                      G T A G C G G C G  | bc: A, 5, gs: , 0


In [113]:
t = 'AAAACGGCCTATTCGGCTATTTCGGC'
p = 'TTCGGC'
bmSteps(p, t)

A A A A C G G C C T A T T C G G C T A T T T C G G C|
T T C G G C                                        | bc: G, 0, gs: , 0
  T T C G G C                                      | bc: G, 0, gs: , 0
    T T C G G C                                    | bc: A, 1, gs: CGGC, 0
        T T C G G C                                | bc: T, 3, gs: , 0
                T T C G G C                        | bc: T, 2, gs: C, 2
                      T T C G G C                  | MATCH
                        T T C G G C                | bc: T, 3, gs: , 0
                                T T C G G C        | bc: T, 3, gs: , 0
                                        T T C G G C| MATCH
